# Imports And Libraries

In [1]:
from square.client import Client
import os
import pandas as pd

from google_sheets import *

# import pandas as pd

# import os
import pickle

# Generic Functions

In [2]:
def get_value(key1,key2):
    try:
        return key1[key2] 
    except:
        return None

# Get Existing Subs

In [3]:
SAMPLE_SPREADSHEET_ID_input = '1K_uXqI1eiguBMr0uH3Tu7xDzePAIi3uAsDrAWukyBq8'
SAMPLE_RANGE_NAME = 'Square Subs!A:G'

exist = get_spread_sheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME)

# Get Data of most Recent Sub

In [4]:
dates = exist[exist['Date'].str.contains('T')]

most_recent_transaction_date = pd.to_datetime(dates['Date'], format='mixed').max().isoformat()

most_recent_transaction_date

'2023-09-04T11:25:25+00:00'

# Get New Subs

In [5]:
client = Client(
    access_token=os.environ['SQUARE_ACCESS_TOKEN'],
    environment='production')


In [6]:
orders_api = client.orders
customers_api = client.customers

In [7]:
most_recent_transaction_date

'2023-09-04T11:25:25+00:00'

In [8]:
body = {
    'location_ids': [
        'SBBJSGR24YHYB'
    ],
    'query': {
        'filter': {
            
            'date_time_filter': {
                'updated_at': {
                    'start_at': f'{most_recent_transaction_date}',
                }
            },
      # "state_filter": {
        # "states": [
        #   "COMPLETED"
        # ]
      # }
       },
    "sort": {
      "sort_field": "UPDATED_AT",
      "sort_order": "DESC"
    }
        
    },
   }

In [9]:
s_orders = orders_api.search_orders(body).body['orders']

len(s_orders)

3

#  Get All Invoices

In [10]:
invoices_api = client.invoices


result = invoices_api.list_invoices(
    'SBBJSGR24YHYB'
)

invoices = pd.DataFrame(result.body['invoices'])

i=1
while result.cursor:
    print(i)
    i+=1
    result = invoices_api.list_invoices(
        'SBBJSGR24YHYB',
        cursor = result.cursor
    )
invoices = pd.concat([invoices, pd.DataFrame(result.body['invoices'])])
invoices


1


,id,version,location_id,payment_requests,primary_recipient,invoice_number,title,description,public_url,next_payment_amount_money,status,timezone,created_at,updated_at,accepted_payment_methods,delivery_method,store_payment_method_enabled,order_id,sale_or_service_date,scheduled_at
0,inv:0-ChCJR2fioaXGo9_zl8L-46s3EIMP,0,SBBJSGR24YHYB,[{'uid': 'b22432fe-88ac-4614-8799-65d8f5b16e33...,"{'customer_id': 'RERYVQH9HD433CNQXNWRVSX78R', ...",000214,,,https://squareup.com/pay-invoice/inv:0-ChCJR2f...,"{'amount': 5000, 'currency': 'GBP'}",UNPAID,Europe/London,2023-09-04T12:30:28Z,2023-09-04T12:30:28Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,NaN,NaN,NaN
1,inv:0-ChA2r0i0S8yIaH96_UWC1lsKEIMP,2,SBBJSGR24YHYB,[{'uid': 'dcf691bf-a9cc-4ade-8fb4-dbec0c2a194c...,"{'customer_id': '4H65N3VWBX1ND35G52Z7V4C450', ...",2023-09-03_3_4H65N3VWBX1ND35G52Z7V4C450_1,Match Subs 02 Sep vs Westminister Casuals,Match Subs 02 Sep vs Westminister Casuals,https://squareup.com/pay-invoice/inv:0-ChA2r0i...,"{'amount': 1000, 'currency': 'GBP'}",UNPAID,Europe/London,2023-09-03T15:37:42Z,2023-09-04T10:00:02Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,PWPOUhRPEGNFx20cBhPA6Oygp7CZY,2023-09-03,NaN
2,inv:0-ChD2EZdeQvddtkkmAlM2wicSEIMP,3,SBBJSGR24YHYB,[{'uid': 'f5675b25-9ed4-491f-83e7-d2fc247492ec...,"{'customer_id': '9WW4E1EQFMX5H2RJM267CNNR1M', ...",2023-09-03_3_9WW4E1EQFMX5H2RJM267CNNR1M_1,Match Subs 02 Sep vs Westminister Casuals,Match Subs 02 Sep vs Westminister Casuals,https://squareup.com/pay-invoice/inv:0-ChD2EZd...,NaN,PAID,Europe/London,2023-09-03T15:37:41Z,2023-09-04T11:25:23Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,ZLfm2FvJNVRvMfdkRsl2SbE1OOPZY,2023-09-03,NaN
3,inv:0-ChAlN9UebQyUTPBjIPi_7iEeEIMP,3,SBBJSGR24YHYB,[{'uid': '3d7e4f9c-5cd6-492e-84bb-07501ddf846a...,"{'customer_id': 'TC0C4DZ3Y217B8Y8A1EZ0PV4F0', ...",2023-09-03_1_TC0C4DZ3Y217B8Y8A1EZ0PV4F0_1,Match Subs 02 Sep vs Old Tenisonians,Match Subs 02 Sep vs Old Tenisonians,https://squareup.com/pay-invoice/inv:0-ChAlN9U...,NaN,PAID,Europe/London,2023-09-03T14:51:38Z,2023-09-04T13:42:25Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,L8jU3V7OqgFJeWtNdihvOo7JPcBZY,2023-09-03,NaN
4,inv:0-ChDfnGPqTS_5UPCqZmdN6jn4EIMP,2,SBBJSGR24YHYB,[{'uid': 'd3a764b3-9d7c-4c38-bac8-16247920d699...,"{'customer_id': 'BYJH2ZRDSH122PJHGD2CWW0ZRR', ...",2023-09-03_1_BYJH2ZRDSH122PJHGD2CWW0ZRR_1,Match Subs 02 Sep vs Old Tenisonians,Match Subs 02 Sep vs Old Tenisonians,https://squareup.com/pay-invoice/inv:0-ChDfnGP...,"{'amount': 1000, 'currency': 'GBP'}",UNPAID,Europe/London,2023-09-03T14:51:36Z,2023-09-04T10:00:02Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,jwJjkscTVYaMLPcohxbhDFU87gCZY,2023-09-03,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,inv:0-ChAxyXkM34z5Rr_hR1qcghr3EIMP,2,SBBJSGR24YHYB,[{'uid': 'ec717fdb-a9fd-431a-9ee6-69e76da24af1...,"{'customer_id': '0HTAK3MQ28S69A5KBCTZ8HKHC0', ...",000008,,,https://squareup.com/pay-invoice/inv:0-ChAxyXk...,NaN,PAID,Europe/London,2023-06-01T17:45:39Z,2023-06-05T06:19:45Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,J8M3Q8SQJERA16Tyz8YWTyueV,NaN,NaN
78,inv:0-ChAMb2fmrMLIoY-QsyhO607cEIMP,4,SBBJSGR24YHYB,[{'uid': 'dce09c3e-dc3c-4b3b-a399-313acb7090dd...,"{'customer_id': '4KEVYTZKFD01B4NE8KA0N1X8Q4', ...",000007,,,https://squareup.com/pay-invoice/inv:0-ChAMb2f...,NaN,PAID,Europe/London,2023-06-01T17:45:00Z,2023-06-02T11:03:53Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,TBKQ9UN9lcEcTA52W7S6HcpeV,NaN,NaN
79,inv:0-ChBSYVNkiKpyEQslJ1DOvhaHEIMP,3,SBBJSGR24YHYB,[{'uid': '58ed6fef-84c4-4a40-8323-94be5b2f0315...,"{'customer_id': 'C3S5WM4AS12RD0CZQ6DS8QBDN4', ...",000005,,,https://squareup.com/pay-invoice/inv:0-ChBSYVN...,NaN,PAID,Europe/London,2023-06-01T17:26:29Z,2023-06-01T17:28:31Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,False,N0B8KtXp70Qg6VikEGxGHxveV,NaN,NaN
80,inv:0-ChCP7LR57_FEOKwBMVQB0odAEIMP,2

# Format new subs and get customer names

In [11]:
orders_format = []
for i, order in enumerate(s_orders[:]):
    
    print(i, '/', len(s_orders))
    new_order = {}
    
    new_order['id']=get_value(order,'id')
    new_order['created_at']=get_value(order,'updated_at')
    
    new_order['customer_id']=get_value(order,'customer_id')
    
    new_order['state']=get_value(order,'state')

    new_order['tenders']=get_value(order,'tenders')

    if not new_order['customer_id']:
        customer_ids = []
        try:
            for tender in order['tenders']:
                if get_value(tender,'customer_id'):
                    customer_ids.append(get_value(tender,'customer_id'))
            new_order['customer_id'] = customer_ids[0]
        except:
            pass

    if new_order['customer_id']:
        customer = customers_api.retrieve_customer(new_order['customer_id'])
    
        try:
            new_order['first_name'] = get_value(customer.body['customer'],'given_name')
            new_order['last_name'] = get_value(customer.body['customer'],'family_name')
        except:
            print(customer.body)
    else:
        pass
        invoice = invoices[invoices['order_id']==new_order['id']]
        
        
        if len(invoice)>0:
            cust_id = list(invoice['primary_recipient'])[0]['customer_id']
            customer = customers_api.retrieve_customer(cust_id)
            try:
                new_order['first_name'] = get_value(customer.body['customer'],'given_name')
                new_order['last_name'] = get_value(customer.body['customer'],'family_name')
            except:
                print(customer.body)
            
    for item in order['line_items']:
        line_dict = new_order.copy()
        
        line_dict['variation_name']=get_value(item,'variation_name')
        line_dict['item_name']=get_value(item,'name')
        line_dict['note']=get_value(item,'note')
        line_dict['total_money']=item['total_money']['amount']/100
        orders_format.append(line_dict) 


orders_df = pd.DataFrame(orders_format)

0 / 3
1 / 3
2 / 3


# Get Correct columns and filter out unwanted orders

In [12]:
orders_df['customer_name']= orders_df['first_name'] + ' ' + orders_df['last_name']
match_fees_export = orders_df[(orders_df['state'].isin(['COMPLETED','OPEN'])) & (orders_df['tenders'].notnull())][['id', 'created_at', 'variation_name','item_name', 'customer_name', 'total_money', 'note']]
# match_fees_export = match_fees_export[match_fees_export['tenders'].notnull()]


match_fees_export = match_fees_export[(match_fees_export['customer_name'].notnull()) | (match_fees_export['note'].notnull())]


match_fees_export = match_fees_export[(match_fees_export['item_name'].fillna('').str.lower().str.contains('sub'))|(match_fees_export['item_name'].fillna('').str.lower().str.contains('training'))]

In [13]:
match_fees_export.columns = exist.columns

In [14]:
match_fees_export

,ID,Date,Cat,Item,Customer,Price,Notes
0,L8jU3V7OqgFJeWtNdihvOo7JPcBZY,2023-09-04T13:42:27.000Z,Full Game,Match Sub,Adam Platten,5.0,None
2,ZLfm2FvJNVRvMfdkRsl2SbE1OOPZY,2023-09-04T11:25:25.000Z,Full Game,Match Sub,Denver Lalonde,10.0,None


# Only upload new transactions

In [15]:
mg = pd.merge(left=match_fees_export, right=exist, how='left', on='ID', suffixes=['', '_old'])
not_in = mg[mg['Date_old'].isnull()][exist.columns]
len(not_in)

1

In [16]:
not_in

,ID,Date,Cat,Item,Customer,Price,Notes
0,L8jU3V7OqgFJeWtNdihvOo7JPcBZY,2023-09-04T13:42:27.000Z,Full Game,Match Sub,Adam Platten,5.0,None


In [17]:
not_in.fillna('NA', inplace=True)

# Add to Google sheet

In [18]:
append_to_sheet('Square Subs', SAMPLE_SPREADSHEET_ID_input, not_in)